<div class="alert alert-block alert-warning">

In order to use Llama2, you need access from Meta. We suggest following the steps in this [article](https://medium.com/@yashambekar1804/getting-started-with-llma2-accessing-llama2-70b-model-and-obtaining-hugging-face-api-token-and-8773e19bda26). 

# HR Chatbot using Retrieval Augmented Generation

Many companies aim to make HR documents more accessible to employees and contractors. A recent promising approach is building a RAG system on top of HR documents. This is because it
- provides a unique response to any query,
- reduces hallucinations as the answer has to be grounded in the retrieved context,
- makes the process highly scalable due to automatic question-answering.

However, this solution possesses its unique challenges. For example
- keeping the knowledgebase consistent and up-to-date,
- running LLMs efficiently,
- ensuring the generated results are correct and aligned with company communication guidelines,

In our case, we have three HR policy sources for our hypothetical company.
1. An older HR policy, which contains our maternity leave policy and details on manager responsibilities.
1. A more recent update, that contains inaccurate information on management responsibilities - but one that also contains unique information on paternity leave.
1. A newer policy document that contains updated information about management responsibilities, correcting the mistakes of the previous modernizing attempt - alongside some other HR policy information.

A good system will be able to:
- provide relevant information on maternity leave (only covered in the old document),
- synthesize contradicting information and only present to us the correct ones

Regarding synthesizing information, there are contradictions between the documents pertaining to management responsibilities. We are going to use the 
- creation_date of the policy,
- and the usefulness score of each paragraph

as a proxy to know for similar documents with 

- similar information but different wording, and some important parts slightly altered, or
- also talking about seemingly the same topic, but without conveying useful information

which one is more relevant to our query.

## Boilerplate

### Installation

In [1]:
%pip install superlinked==18.7.1

### Imports and constants

In [2]:
from datetime import datetime, timedelta
import os

import altair as alt
import pandas as pd
from torch import float16
from transformers import AutoTokenizer, pipeline
from superlinked import framework as sl

alt.renderers.enable(sl.get_altair_renderer())
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", 1000)
START_OF_2024_TS = int(datetime(2024, 1, 2).timestamp())
EXECUTOR_DATA = {sl.CONTEXT_COMMON: {sl.CONTEXT_COMMON_NOW: START_OF_2024_TS}}
TOP_N = 8

## Load data

In [3]:
text_df = pd.read_csv("https://storage.googleapis.com/superlinked-notebook-hr-knowledgebase/hr_rag_policies.csv")
text_df.head()

,index,body,creation_date,usefulness
0,0,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",1672527600,0.555771
1,1,"Management should factor in undue pressures and potential fraud scenarios when devising incentives and delineating roles, duties, and authorizations.",1672527600,0.310004
2,2,"Management must guarantee that information security obligations are conveyed to individuals, either through written job outlines, policies, or alternative documented means that are regularly updated and managed accurately. Compliance with information security protocols and the fulfillment of related duties should form part of the performance evaluation process in all cases.",1672527600,0.410012
3,3,"Each quarter, all employees of <Company Name> will undergo a performance evaluation encompassing an appraisal of job performance, competency within their respective roles, adherence to company regulations and ethical standards, and attainment of role-specific goals.",1672527600,0.389284
4,4,"To sustain a high level of security awareness, the organization will disseminate security updates and communications to its personnel occasionally via various communication platforms as necessary.",1672527600,0.594588


Our dataset contains documents from all 3 sources, with the following columns:
- index,
- body: text of the HR policy paragraph,
- creation_date: utc timestamp when the policy was issued,
- usefulness: the ratio of people deeming the paragraph useful when prompted.

<style>
.table_component {
    overflow: auto;
    width: 100%;
}

.table_component table {
    border: 1px solid #dededf;
    height: 100%;
    width: 100%;
    table-layout: fixed;
    border-collapse: collapse;
    border-spacing: 1px;
    text-align: left;
}

.table_component caption {
    caption-side: top;
    text-align: left;
}

.table_component th {
    border: 1px solid #dededf;
    background-color: #eceff1;
    color: #000000;
    padding: 5px;
}

.table_component td {
    border: 1px solid #dededf;
    background-color: #ffffff;
    color: #000000;
    padding: 5px;
}
</style>
<div class="table_component" role="region" tabindex="0">
<table>
    <caption>
        <p><b>Breakdown of policy documents</b></p>
    </caption>
    <thead>
        <tr>
            <th><b>Document</b></th>
            <th><b>Relevance</b></th>
            <th><b>Recency</b></th>
            <th><b>Usefulness</b></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Old document<br></td>
            <td>Management responsibilities, maternity leave<br></td>
            <td>Low<br></td>
            <td>Mid-high<br></td>
        </tr>
        <tr>
            <td>Recent, but low quality</td>
            <td>Management responsibilities, paternal leave</td>
            <td>High</td>
            <td>Low, but high for paternity leave</td>
        </tr>
        <tr>
            <td>Recent, fixed</td>
            <td>Management responsibilities</td>
            <td>High</td>
            <td>High</td>
        </tr>
    </tbody>
</table>
</div>

## Set up Superlinked for Retrieval

In [4]:
# typed schema to describe our inputs
class ParagraphSchema(sl.Schema):
    body: sl.String
    created_at: sl.Timestamp
    usefulness: sl.Float
    id: sl.IdField

In [5]:
paragraph = ParagraphSchema()

In [6]:
%%capture
# relevance space will encode our knowledgebase utilizing chunking to control the granularity of information
relevance_space = sl.TextSimilaritySpace(
    text=sl.chunk(paragraph.body, chunk_size=100, chunk_overlap=20),
    model="sentence-transformers/all-mpnet-base-v2",
)
# recency has a periodtime to differentiate between the document created at the beginning of this year and last year
recency_space = sl.RecencySpace(
    timestamp=paragraph.created_at,
    period_time_list=[sl.PeriodTime(timedelta(days=300))],
    negative_filter=-0.25,
)
# usefulness is a ratio, so the relevant range is [0-1], and we prefer higher numbers
usefulness_space = sl.NumberSpace(number=paragraph.usefulness, min_value=0.0, max_value=1.0, mode=sl.Mode.MAXIMUM)

In [7]:
paragraph_index = sl.Index([relevance_space, recency_space, usefulness_space])
paragraph_parser = sl.DataFrameParser(paragraph, mapping={paragraph.id: "index", paragraph.created_at: "creation_date"})
source: sl.InMemorySource = sl.InMemorySource(paragraph, parser=paragraph_parser)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index], context_data=EXECUTOR_DATA)
app = executor.run()
source.put([text_df])

In [8]:
# our simple query will make a search term possible, and gives us the opportunity to weight input aspects
# relevance, recency and usefulness against each other
knowledgebase_query = (
    sl.Query(
        paragraph_index,
        weights={
            relevance_space: sl.Param("relevance_weight"),
            recency_space: sl.Param("recency_weight"),
            usefulness_space: sl.Param("usefulness_weight"),
        },
    )
    .find(paragraph)
    .similar(relevance_space, sl.Param("search_query"))
    .limit(sl.Param("limit"))
)

## Perform retrieval

In [9]:
def present_result(result: sl.Result) -> pd.DataFrame:
    """A small helper function to present our query results"""
    df = result.to_pandas()
    df["date_created"] = [datetime.fromtimestamp(ts).date() for ts in df["created_at"]]
    df.drop("created_at", axis=1, inplace=True)
    return df

In [10]:
initial_query_text: str = "What should management monitor?"

First, let's do a simple retrieval based only on text similarity. We can see that the recency-, and usefulness weight is set to 0, so the document creation dates or usefulness scores are not affecting our results.

In [11]:
only_relevance_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0,
    usefulness_weight=0,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(only_relevance_result)

,body,usefulness,id,similarity_score,rank,date_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",0.469966,23,0.545108,0,2024-01-01
1,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",0.555771,0,0.522990,1,2023-01-01
2,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",0.830746,24,0.507148,2,2024-01-01
3,"Management must guarantee that information security obligations are conveyed to individuals, either through written job outlines, policies, or alternative documented means that are regularly updated and managed accurately. Compliance with information security protocols and the fulfillment of related duties should form part of the performance evaluation process in all cases.",0.410012,2,0.491230,3,2023-01-01
4,"Management should factor in undue pressures and potential fraud scenarios when devising incentives and delineating roles, duties, and authorizations.",0.310004,1,0.489609,4,2023-01-01
5,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",0.551468,25,0.489070,5,2024-01-01
6,"Management should factor in undue pressures and potential fraud scenarios when devising splorps and delineating roles, duties, and authorizations.",0.030283,34,0.468849,6,2023-12-23
7,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0.665170,17,0.466823,7,2024-01-01


Let's observe the difference between elements with ids 23 and 0 respectively. These documents essentially say the same thing, except that the old document (id 0) prescribes a bi-annual, while the fresh one (id 23) an annual review on policies and procedures. Also notice that an element (id 34) with very low usefulness score appears it the top 10 documents when only accounting for relevance. Let's see what happens if we upweight recency!

In [12]:
mild_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.10,
    usefulness_weight=0,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(mild_recency_result)

,body,usefulness,id,similarity_score,rank,date_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",0.469966,23,0.391288,0,2024-01-01
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",0.830746,24,0.364579,1,2024-01-01
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",0.551468,25,0.351860,2,2024-01-01
3,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",0.555771,0,0.342974,3,2023-01-01
4,"Management should factor in undue pressures and potential fraud scenarios when devising splorps and delineating roles, duties, and authorizations.",0.030283,34,0.340918,4,2023-12-23
5,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0.665170,17,0.336207,5,2024-01-01
6,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",0.714163,21,0.325388,6,2024-01-01
7,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",0.781372,28,0.325023,7,2024-01-01


Now, note how the relative position of element 23 compared to 0 improved as it received a score boost for its recency. It is a useful feature that enables us to differentiate between similar documents based on their freshness - let's boost that and see if we can have only recent, not outdated, information in our results.

In [13]:
normal_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.4,
    usefulness_weight=0,
    search_query=initial_query_text,
    limit=TOP_N,
)

norm_recency_result_df = present_result(normal_recency_result)
norm_recency_result_df

,body,usefulness,id,similarity_score,rank,date_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",0.469966,23,0.386812,0,2024-01-01
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",0.830746,24,0.361890,1,2024-01-01
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",0.551468,25,0.350022,2,2024-01-01
3,"Management should factor in undue pressures and potential fraud scenarios when devising splorps and delineating roles, duties, and authorizations.",0.030283,34,0.349009,3,2023-12-23
4,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0.665170,17,0.335416,4,2024-01-01
5,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",0.714163,21,0.325320,5,2024-01-01
6,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",0.781372,28,0.324980,6,2024-01-01
7,"Management must guarantee that employee security obligations are conveyed to individuals, either through written job zizzles, policies, or alternative documented means that are regularly updated and managed accurately. Compliance with employee bamboozle protocols and the fulfillment of related duties should form part of the performance evaluation process in all cases.",0.044163,35,0.323444,7,2023-12-23


Yes, we can! Now that all results are fresh, the outdated counterparts were removed from the result set. This way we can supply nice and clean context to the generation model.

Nevertheless, we still have the usefulness issue. Observe the interesting dynamics here between documents with IDs 24 and 35 - doc 24 softened the requirement to "wherever applicable" from "in all cases". Also, doc 34 and 35 are deemed useless by the employees - looking at the text it is easy to understand why. We need to incorporate usefulness into our search, as recency is not able to differentiate between these two documents, and they are both relevant to our query.

In [14]:
normal_recency_usefulness_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.4,
    usefulness_weight=0.5,
    search_query=initial_query_text,
    limit=TOP_N,
)

normal_recency_usefulness_result_df = present_result(normal_recency_usefulness_result)
normal_recency_usefulness_result_df

,body,usefulness,id,similarity_score,rank,date_created
0,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",0.830746,24,0.502577,0,2024-01-01
1,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",0.781372,28,0.469588,1,2024-01-01
2,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",0.714163,21,0.459939,2,2024-01-01
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0.665170,17,0.458655,3,2024-01-01
4,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",0.469966,23,0.450072,4,2024-01-01
5,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",0.551468,25,0.444445,5,2024-01-01
6,"The skills and competence of employees and contractors shall be assessed by human resources staff and the hiring manager or his or her designees as part of the hiring process. Required skills and competencies shall be listed in job descriptions and requisitions, and/or aligned with the responsibilities outlined in the Roles and Responsibilities Policy. Competency evaluations may include reference checks, education and certification verifications, technical testing, and interviews.",0.762619,20,0.413454,6,2024-01-01
7,"Employees and third-parties who violate <Company Name> information security policies shall be subject to the <Company Name> progressive disciplinary process, up to and including termination of employment or contract.",0.763208,30,0.382965,7,2024-01-01


Giving some emphasis to usefulness eradicated all the seemingly irrelevant docs from our results!

It is also important to see that the same weights produce relevant, useful results for queries about parental leave — a topic parts of which are only discussed in the older HR document in an useful way (maternity leave) or in the fresh but otherwise useless policy document (paternal leave). We should be able to receive results about both topics.

In [15]:
maternity_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.4,
    usefulness_weight=0.5,
    search_query="What are the companies terms for parental leave?",
    limit=TOP_N,
)

maternity_result_df = present_result(maternity_result)
maternity_result_df

,body,usefulness,id,similarity_score,rank,date_created
0,"[Company Name] offers eligible employees 4 weeks of paid paternity leave, with an option for an additional 4 weeks of unpaid leave, to be taken within 6 months of a child's birth or adoption. Employees must provide at least 4 weeks' notice with relevant documentation. During paid leave, employees receive 100% of their salary and retain their benefits, with job protection ensured upon return. This policy supports new fathers and partners with the necessary time to bond with their child and includes access to flexible working arrangements and counseling services. The policy is reviewed annually and may be amended as needed.",0.812300,51,0.613502,0,2023-12-23
1,[Company Name] is committed to supporting employees during significant life events such as the birth or adoption of a child. Our Maternity Leave Policy aims to provide eligible employees with the necessary time off to bond with their newborn or newly adopted child and to ensure a smooth transition back to work.,0.656872,6,0.433878,1,2023-01-01
2,"This policy applies to all employees of <Company Name>, consultants, contractors and other third-party entities with access to <Company Name> production networks and system resources.",0.764244,18,0.416710,2,2024-01-01
3,"Eligible employees are entitled to up to [XX weeks/months] of paid maternity leave, which includes both prenatal and postnatal leave. Additionally, eligible employees may also take up to [XX weeks/months] of unpaid maternity leave following the conclusion of their paid leave, as provided under the Family and Medical Leave Act (FMLA) and applicable state laws.",0.642192,8,0.415726,3,2023-01-01
4,"Following childbirth or adoption, employees are entitled to [XX weeks/months] of paid maternity leave to recover from childbirth, bond with their newborn or adopted child, and adjust to their new family dynamic. This period of leave is intended to promote the physical and emotional well-being of both the parent and child during the crucial early stages of development.",0.621864,10,0.414399,4,2023-01-01
5,"All full-time and part-time employees are eligible for maternity leave if they have been employed by [Company Name] for at least 12 months and have worked a minimum of 1,250 hours during the 12-month period preceding the commencement of leave. This policy applies equally to biological mothers, adoptive parents, and employees expecting the birth of a child through surrogacy.",0.608694,7,0.408767,5,2023-01-01
6,"[Company Name] is committed to maintaining the confidentiality of employees' maternity leave and pregnancy-related information in accordance with applicable privacy laws. Additionally, [Company Name] prohibits discrimination or retaliation against employees for exercising their rights under this policy or applicable law.",0.664988,14,0.401210,6,2023-01-01
7,"Employee and contractor termination and offboarding processes shall ensure that physical and logical access is promptly revoked in accordance with company SLAs and policies, and that all company issued equipment is returned. Any security or confidentiality agreements which remain valid after termination shall be communicated to the employee or contractor at time of termination.",0.671677,29,0.400823,7,2024-01-01


Yes, our weights work in this case too! We can go on and create our chatbot on top of the retrieval we set up.

# NEW Natural Language Querying

This way we can save the trouble of setting the weights and we can let the system set them based on the detected user intent. Check the feature out in more depth in the corresponding [feature notebook](https://github.com/superlinked/superlinked/blob/main/notebook/feature/natural_language_querying.ipynb)!

In [16]:
# switch to your working OpenAI API key!
openai_config = sl.OpenAIClientConfig(api_key=os.environ["OPEN_AI_API_KEY"], model="gpt-4o")

nlq_knowledgebase_query = knowledgebase_query.with_natural_query(sl.Param("natural_query"), openai_config)

In [17]:
nlq_result = app.query(
    nlq_knowledgebase_query,
    natural_query="recent useful info on management responsibilities",
    limit=TOP_N,
)

nlq_result_df = present_result(nlq_result)
nlq_result_df

,body,usefulness,id,similarity_score,rank,date_created
0,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",0.830746,24,0.552296,0,2024-01-01
1,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",0.781372,28,0.495011,1,2024-01-01
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",0.551468,25,0.479773,2,2024-01-01
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0.665170,17,0.479567,3,2024-01-01
4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",0.714163,21,0.462605,4,2024-01-01
5,"The skills and competence of employees and contractors shall be assessed by human resources staff and the hiring manager or his or her designees as part of the hiring process. Required skills and competencies shall be listed in job descriptions and requisitions, and/or aligned with the responsibilities outlined in the Roles and Responsibilities Policy. Competency evaluations may include reference checks, education and certification verifications, technical testing, and interviews.",0.762619,20,0.461437,5,2024-01-01
6,"This policy applies to all employees of <Company Name>, consultants, contractors and other third-party entities with access to <Company Name> production networks and system resources.",0.764244,18,0.453328,6,2024-01-01
7,"Company policies and information security roles and responsibilities shall be communicated to employees and third-parties at the time of hire or engagement, and employees and contractors are required to formally acknowledge their understanding and acceptance of their security responsibilities. Employees and third-parties with access to company or customer information shall sign an appropriate non-disclosure, confidentiality, and appropriate code-of-conduct agreements. Contractual agreements shall state responsibilities for information security as needed. Employees and relevant third-parties shall follow all <Company Name> information security policies.",0.553113,22,0.450416,7,2024-01-01
8,"Employees and third-parties who violate <Company Name> information security policies shall be subject to the <Company Name> progressive disciplinary process, up to and including termination of employment or contract.",0.763208,30,0.447828,8,2024-01-01
9,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",0.469966,23,0.441084,9,2024-01-01


## Augmentation - formulate query for our LLM for generation

To keep this light and tool independent, we just manually craft our query template based on [LLama2 instructions from HuggingFace](https://huggingface.co/blog/llama2)

In [18]:
context_items_from_retrieval: int = 5
context_text: str = (
    "\n" + "\n".join(f for f in normal_recency_usefulness_result_df["body"].iloc[:context_items_from_retrieval]) + "\n"
)

rag_query = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: {context_text}
QUESTION: {initial_query_text}
[/INST]"""

In [19]:
print(rag_query)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: 
Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be

## Generation - prompt the LLM for text generation to answer our query

In [20]:
# load the 7B parameter Llama model for chat from Huggingface
# for the first run this will download model, for subsequent ones it will load it
model = "meta-llama/Llama-2-7b-chat-hf"

# we use the pure transformers library without popular wrappers
# but LLamaIndex and LangChain can easily be plugged in here as those use transformers under the hood too
tokenizer = AutoTokenizer.from_pretrained(model)
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# we prompt the LLM for text generation with our RAG query
sequences = generation_pipeline(
    rag_query,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: <s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: 
Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities 

## Evaluation of the answer

The results are structured, contain the information present in the context documents, and are tailored to our question. Based on hardware, it will run in a manageable time. 
In 3), we can see that the generated text contains the correct annual term. This is mainly because it was fed the right thing as context information.